In [6]:
import numpy as np

In [1]:
()


hands=["4 of a Kind", "Straight Flush", "Straight", "Flush", "High Card", 
       "1 Pair", "2 Pair", "Royal Flush", "3 of a Kind", "Full House" ]
point=[40, 100, 4, 6, -1, 
       1, 2, 500, 3, 10 ]
_ = [1,2,4,8]
 
def rankPokerHand(hand) :
    cs = [i%13+2 for i in hand]
    ss = [_[i//13] for i in hand]
    v = 0
    i = -1
    o = 0
    s = 1<<cs[0]|1<<cs[1]|1<<cs[2]|1<<cs[3]|1<<cs[4]
    while i<5 :
        if o != 0:
            v += o*((v//o & 15)+1)
        
        o=2**(cs[i]*4)
        i+=1
    v = v % 15 - (3 if ((s/(s&-s) == 31) or (s == 0x403c)) else 1)
    v -= (ss[0] == (ss[1]|ss[2]|ss[3]|ss[4])) * (-5 if (s == 0x7c00) else 1)
    if v == 5:
        pairs = set(cs)
        # collect pairs
        pairs = [f for f in pairs if cs.count(f) == 2]
#         print(pairs)
        if pairs[0] < 6:
            return -1
        elif pairs[0] > 10:
            return 1
        else:
            return 0
    else:
        return point[v]


rankPokerHand([8, 9, 10, 11, 12] ) # Royal Flush
rankPokerHand([2, 3, 4, 5, 6]) # Straight Flush
rankPokerHand([0, 1, 2, 3, 12] ) # Straight Flush
rankPokerHand([6, 19, 32, 45, 7]) # 4 of a Kind
rankPokerHand([5, 18, 31, 7, 20] ) # Full house
rankPokerHand([21, 22, 17, 24, 20] ) # Flush
rankPokerHand([8, 22, 36, 24, 46]) # Straight
rankPokerHand([0, 14, 28, 16, 51]) # Straight
rankPokerHand([2, 15, 28, 6, 20]) # 3 of a Kind
rankPokerHand([6, 19, 35, 7, 20]) # 2 Pair
rankPokerHand([3, 16, 27, 4, 20]) # 1 Pair
rankPokerHand([4, 17, 27, 3, 20]) # 1 Pair
rankPokerHand([8, 21, 27, 3, 20]) # 1 Pair
rankPokerHand([9, 22, 27, 3, 20]) # 1 Pair
rankPokerHand([8, 16, 28, 5, 20]) # High Card

-1

In [44]:
from itertools import combinations
cards = [i for i in range(52)]
comb = combinations(cards,4)
table = list(comb)
print(len(table))
outcome = [] #np.zeros((len(table),3,4,3))
for i in table:
    for j in range(52):
        if j not in i:
            for k in range(4):
                for l in range(3):
                    outcome.append(rankPokerHand(list(i+(j,))))
print(np.mean(np.array(outcome)))
print(len(outcome))

270725
-0.2890587004032382
12994800


In [32]:
from collections import namedtuple


suits = "HDCS".split()
faces = "2,3,4,5,6,7,8,9,T,J,Q,K,A"
face = faces.split(',')
highfaces = "6,7,8,9,T,J,Q,K,A"
highface = highfaces.split(',')
class Card(namedtuple('Card', 'face, suit')):
    def __repr__(self):
        return ''.join(self)
    
def character_frequency(s):
    freq = {}
    for i in s:
        if i in freq:
            freq[i] += 1
        else:
            freq[i] = 1
    return freq

def royal_flush(hand):
    royalface = "TJQKA"
    # sort the cards based on the face rank of each card
    ordered = sorted(hand, key=lambda card: (faces.index(card.face), card.suit))

    first_card = ordered[0]
    other_cards = ordered[1:]

    # check if all are of the same suit
    if all(first_card.suit == card.suit for card in other_cards):
        # check if they are in sequential order
        # compare the ordered faces substring with the face list (which is converted to string)
        if ''.join(card.face for card in ordered) in royalface:
            return 500
    return False

def straight_flush(hand):
    # sort the cards based on the face rank of each card
    ordered = sorted(hand, key=lambda card: (faces.index(card.face), card.suit))

    first_card = ordered[0]
    other_cards = ordered[1:]

    # check if all are of the same suit
    if all(first_card.suit == card.suit for card in other_cards):
        # check if they are in sequential order
        # compare the ordered faces substring with the face list (which is converted to string)
        if ''.join(card.face for card in ordered) in ''.join(face):
            return 100
    return False

def four_of_a_kind(hand):
    allfaces = [f for f,s in hand]
    
    # create a unique set of ranks
    uniqueRanks = set(allfaces)

    # if there are more than 2 ranks, it's not four of a kind
    if len(uniqueRanks) != 2:
        return False

    for f in uniqueRanks:
        # if there are 4 faces, it is four of a kind
        if allfaces.count(f) == 4:
            uniqueRanks.remove(f)
            return 40

    return False

def full_house(hand):
    allfaces = [f for f,s in hand]

    rankFrequency = character_frequency(allfaces)

    # if there are 2 types of ranks and there's a card with 1 pair and 3 of a kind
    if len(rankFrequency) == 2 and (rankFrequency.values()[0] == 2 and rankFrequency.values()[1] == 3):
        return 10

    return False

def flush(hand):
    allfaces = [f for f,s in hand]

    first_card = hand[0]
    other_cards = hand[1:]

    if all(first_card.suit == card.suit for card in other_cards):
        return 6

    return False

def straight(hand):
    ordered = sorted(hand, key=lambda card: (faces.index(card.face), card.suit))
    if ''.join(card.face for card in ordered) in ''.join(face):
        return 4
    return False;

def three_of_a_kind(hand):
    allfaces = [f for f,s in hand]

    uniqueRanks = set(allfaces)

    if len(uniqueRanks) != 3:
        return False

    for f in uniqueRanks:
        if allfaces.count(f) == 3:
            uniqueRanks.remove(f)
            return 3

    return False;

def two_pair(hand):
    allfaces = [f for f,s in hand]
    allftypes = set(allfaces)
    
    # collect pairs
    pairs = [f for f in allftypes if allfaces.count(f) == 2]
    
    # if there are more than two pairs
    if len(pairs) != 2:
        return False

    p1, p2 = pairs
    # get the difference using sets
    other_cards = [(allftypes - set(pairs)).pop()]
    return 2

def one_pair(hand):
    allfaces = [f for f,s in hand]
    allftypes = set(allfaces)
    # collect pairs
    pairs = [f for f in allftypes if allfaces.count(f) == 2]
    # if there's more than one pair
    if len(pairs) != 1:
        return False
    if pairs[0] not in highface:
        return False
    allftypes.remove(pairs[0])
    return 1

def high_card(hand):
    return -1

def create_hand_tuple(cards = "5D 8C 9S JS AC"):
    hand = []
    for card in cards.split():
        face, suit = card[:-1], card[-1]
        hand.append(Card(face, suit))
    return hand;

# functions
handrankorder = (royal_flush,straight_flush,four_of_a_kind,full_house,
                flush,straight,three_of_a_kind,two_pair,
                one_pair,high_card)

def determine_rank(cards):
    hand = create_hand_tuple(cards)
    for ranker in handrankorder:
        rank = ranker(hand)

        if rank:
            break
    return rank

In [33]:
determine_rank("6D 6C 7S 8S 9C")

1